In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
train_dir = "dataset/train"
valid_dir = "dataset/validation"
test_dir = "dataset/test"
num_classes = 10

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 837 images belonging to 10 classes.
Found 99 images belonging to 10 classes.
Found 97 images belonging to 10 classes.


In [10]:
import tensorflow as tf

tf.random.set_seed(42)
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=[224,224,3]),
  tf.keras.layers.Dense(256, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(10, activation="softmax")
])


In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 49152)             0         
                                                                 
 dense_6 (Dense)             (None, 256)               12583168  
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 10)                1290      
                                                                 
Total params: 12,617,354
Trainable params: 12,617,354
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/30
26/26 [==============================] - 47s 2s/step - loss: 21.9117 - accuracy: 0.1106 - val_loss: 4.6247 - val_accuracy: 0.2292
Epoch 2/30
26/26 [==============================] - 47s 2s/step - loss: 5.5741 - accuracy: 0.1950 - val_loss: 3.2435 - val_accuracy: 0.2500
Epoch 3/30
26/26 [==============================] - 48s 2s/step - loss: 3.3315 - accuracy: 0.2447 - val_loss: 3.7034 - val_accuracy: 0.2396
Epoch 4/30
26/26 [==============================] - 57s 2s/step - loss: 3.0373 - accuracy: 0.2410 - val_loss: 2.2550 - val_accuracy: 0.3021
Epoch 5/30
26/26 [==============================] - 50s 2s/step - loss: 2.4720 - accuracy: 0.2944 - val_loss: 2.4843 - val_accuracy: 0.2812
Epoch 6/30
26/26 [==============================] - 47s 2s/step - loss: 2.3786 - accuracy: 0.2832 - val_loss: 2.8430 - val_accuracy: 0.2500
Epoch 7/30
26/26 [==============================] - 55s 2s/step - loss: 2.6038 - accuracy: 0.3143 - val_loss: 1.9477 - val_accuracy: 0.4375
Epoch 8/30
26/26 [=

KeyboardInterrupt: 

In [ ]:
history_df = pd.DataFrame(history.history)
history_df

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["train", "valid"])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["train", "valid"])
plt.show()

In [ ]:
preds = model.predict(test_generator)

In [ ]:
y_pred = np.argmax(preds, axis=1)

In [ ]:
for k, v in test_generator.class_indices.items():
    print(v, ":", k)

In [ ]:
pred_classes = {
    0 : "apple",
    1 : "banana",
    2 : "grapes",
    3 : "kiwi",
    4 : "mango",
    5 : "orange",
    6 : "pear",
    7 : "pineapple",
    8 : "pomegranate",
    9 : "watermelon"
}

In [ ]:
y_pred = list(map(lambda x: pred_classes[x], y_pred))

In [ ]:
y_test = list(map(lambda x: pred_classes[x], test_generator.classes))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y_test))